# Quora Question Pair with LSTM

In [0]:
import numpy as np 
import pandas as pd 
import os
import string
import re
import numpy as np
from collections import Counter
from collections import defaultdict

from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence 

## Dataset

In [2]:
from google.colab import drive
drive.mount('/data')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /data


In [3]:
cd ../data/'My Drive'/data

/data/My Drive/data


In [112]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
train_df = train_df.fillna('None')
test_df = test_df.fillna('None')

## Cleaning the text

In [0]:
# from https://www.kaggle.com/lystdo/lstm-with-word2vec-embeddings
def text_to_wordlist(text, remove_stopwords=False, stem_words=False):
    # Clean the text, with the option to remove stopwords and to stem words.
    
    # Convert words to lower case and split them
    text = text.lower().split()

    # Optionally, remove stop words
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        text = [w for w in text if not w in stops]
    
    text = " ".join(text)

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    
    # Optionally, shorten words to their stems
    if stem_words:
        text = text.split()
        stemmer = SnowballStemmer('english')
        stemmed_words = [stemmer.stem(word) for word in text]
        #text = " ".join(stemmed_words)
    
    # Return a list of words
    text = text.split()
    return(text)


In [0]:
def get_vocab(list_of_content):
    """Computes Dict of counts of words.
    
    Computes the number of times a word is on a document.
    """
    vocab = defaultdict(float)
    for content in list_of_content:
        for line in content:
            line = text_to_wordlist(line.strip())
            words = set(line)
            for word in words:
                vocab[word] += 1
    return vocab       

## Split train and test

In [0]:
questions_cols=['question1','question2']

In [0]:
X = train_df[questions_cols]
Y = train_df['is_duplicate']

X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=42)

In [0]:
X_test = test_df[questions_cols]

In [0]:
X_train1, X_train2 = np.array(X_train.question1), np.array(X_train.question2)
X_val1, X_val2 = np.array(X_val.question1), np.array(X_val.question2)

In [0]:
X_test1, X_test2 = np.array(X_test.question1), np.array(X_test.question2)

In [19]:
x_len = np.array([len(x.split()) for x in X_train2])
np.percentile(x_len, 95) # let set the max sequence len to N=22

24.0

### Computing vocab2index

In [0]:
# getting vocab from training sets
word_count = get_vocab([X_train1,X_train2])

In [21]:
len(word_count.keys())

78051

## Use pretrained word embeddings


In [22]:
! head -2 'glove.6B.50d.txt'

the 0.418 0.24968 -0.41242 0.1217 0.34527 -0.044457 -0.49688 -0.17862 -0.00066023 -0.6566 0.27843 -0.14767 -0.55677 0.14658 -0.0095095 0.011658 0.10204 -0.12792 -0.8443 -0.12181 -0.016801 -0.33279 -0.1552 -0.23131 -0.19181 -1.8823 -0.76746 0.099051 -0.42125 -0.19526 4.0071 -0.18594 -0.52287 -0.31681 0.00059213 0.0074449 0.17778 -0.15897 0.012041 -0.054223 -0.29871 -0.15749 -0.34758 -0.045637 -0.44251 0.18785 0.0027849 -0.18411 -0.11514 -0.78581
, 0.013441 0.23682 -0.16899 0.40951 0.63812 0.47709 -0.42852 -0.55641 -0.364 -0.23938 0.13001 -0.063734 -0.39575 -0.48162 0.23291 0.090201 -0.13324 0.078639 -0.41634 -0.15428 0.10068 0.48891 0.31226 -0.1252 -0.037512 -1.5179 0.12612 -0.02442 -0.042961 -0.28351 3.5416 -0.11956 -0.014533 -0.1499 0.21864 -0.33412 -0.13872 0.31806 0.70358 0.44858 -0.080262 0.63003 0.32111 -0.46765 0.22786 0.36034 -0.37818 -0.56657 0.044691 0.30392


In [0]:
def loadGloveModel(gloveFile="glove.6B.50d.txt"):
    """ Loads word vectors into a dictionary."""
    f = open(gloveFile,'r')
    word_vecs = {}
    for line in f:
        splitLine = line.split()
        word = splitLine[0]
        word_vecs[word] = np.array([float(val) for val in splitLine[1:]])
    return word_vecs

In [0]:
word_vecs = loadGloveModel()

In [25]:
print(len(word_vecs.keys()), len(word_count.keys()))

400000 78051


In [0]:
def delete_rare_words(word_vecs, word_count, min_df=4):
    """ Deletes rare words from word_count
    
    Deletes words from word_count if they are not in word_vecs
    and don't have at least min_df occurrencies in word_count.
    """
    words_delete = []
    for word in word_count:
        if word_count[word] < min_df and word not in word_vecs:
            words_delete.append(word)
    for word in words_delete: word_count.pop(word)
    return word_count

In [0]:
def create_embedding_matrix(word_vecs, word_count, min_df=4, emb_size=50):
    """Creates embedding matrix from word vectors. """
    word_count = delete_rare_words(word_vecs, word_count, min_df)
    V = len(word_count.keys()) + 2
    vocab2index = {}
    W = np.zeros((V, emb_size), dtype="float32")
    vocab = ["", "UNK"]
    # adding a vector for padding
    W[0] = np.zeros(emb_size, dtype='float32')
    # adding a vector for rare words 
    W[1] = np.random.uniform(-0.25, 0.25, emb_size)
    vocab2index["UNK"] = 1
    i = 2
    for word in word_count:
        if word in word_vecs:
            W[i] = word_vecs[word]
            vocab2index[word] = i
            vocab.append(word)
            i += 1
        else:
            W[i] = np.random.uniform(-0.25,0.25, emb_size)
            vocab2index[word] = i
            vocab.append(word)
            i += 1   
    return W, np.array(vocab), vocab2index

In [0]:
pretrained_weight, vocab, vocab2index = create_embedding_matrix(word_vecs, word_count)

In [29]:
len(pretrained_weight)

58626

## Dataset

In [0]:
def encode_sentence(sentence, vocab2index, N=22, padding_start=True):
    x = text_to_wordlist(sentence)
    enc = np.zeros(N, dtype=np.int32)
    enc1 = np.array([vocab2index.get(w, vocab2index["UNK"]) for w in x])
    l = min(N, len(enc1))
    if padding_start:
        enc[:l] = enc1[:l]
    else:
        enc[N-l:] = enc1[:l]
    return enc

In [0]:
class QuoraDataset(Dataset):
    def __init__(self, X1,X2, y):
        self.x1 = [encode_sentence(X,vocab2index) for X in X1]
        self.x2 = [encode_sentence(X,vocab2index) for X in X2]
        self.y = np.array(y)
    
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return self.x1[idx], self.x2[idx], self.y[idx]
    
train_ds = QuoraDataset(X_train1, X_train2, Y_train)
valid_ds = QuoraDataset(X_val1, X_val2, Y_val)


In [0]:
class QuoraDatasetTest(Dataset):
    def __init__(self, X1,X2):
        self.x1 = [encode_sentence(X,vocab2index) for X in X1]
        self.x2 = [encode_sentence(X,vocab2index) for X in X2]
    
    def __len__(self):
        return len(self.x1)
    
    def __getitem__(self, idx):
        return self.x1[idx], self.x2[idx]

test_ds = QuoraDatasetTest(X_test1, X_test2)

In [0]:
class GRUModel(torch.nn.Module) :
    def __init__(self, vocab_size, embedding_dim, hidden_dim, glove_weights=None) :
        super(GRUModel,self).__init__()
        self.hidden_dim = hidden_dim
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        if glove_weights is not None:
            self.embeddings.weight.data.copy_(torch.from_numpy(glove_weights))
            self.embeddings.weight.requires_grad = False ## freeze embeddings
            
        self.gru = nn.GRU(embedding_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, 1)
        self.dropout = nn.Dropout(0.5)
        
    def forward(self, x1, x2):
        x = torch.cat([x1, x2], dim=1)
        x = self.embeddings(x)
        x = self.dropout(x)
        out_pack, ht = self.gru(x)
        return self.linear(ht[-1])

In [0]:
def save_model(m, p): torch.save(m.state_dict(), p)
    
def load_model(m, p): m.load_state_dict(torch.load(p))

In [0]:
def cosine_segment(start_lr, end_lr, iterations):
    i = np.arange(iterations)
    c_i = 1 + np.cos(i*np.pi/iterations)
    return end_lr + (start_lr - end_lr)/2 *c_i

def get_cosine_triangular_lr(max_lr, iterations, div_start=5, div_end=5):
    min_start, min_end = max_lr/div_start, max_lr/div_end
    iter1 = int(0.3*iterations)
    iter2 = iterations - iter1
    segs = [cosine_segment(min_start, max_lr, iter1), cosine_segment(max_lr, min_end, iter2)]
    return np.concatenate(segs)

In [0]:
def update_optimizer(optimizer, lr):
    for i, param_group in enumerate(optimizer.param_groups):
        param_group["lr"] = lr

In [0]:
def train_epocs(model, optimizer, train_dl, val_dl, max_lr = 0.05, epochs=10):
    iterations = epochs*len(train_dl)
    idx = 0
    best_val_loss = 100
    lrs = get_cosine_triangular_lr(max_lr, iterations)
    for i in range(epochs):
        model.train()
        sum_loss = 0.0
        total = 0
        for x1, x2, y in train_dl:
            update_optimizer(optimizer, lrs[idx])
            x1 = x1.long().cuda()
            x2 = x2.long().cuda()
            y = y.float().cuda()
            y_pred = model(x1, x2)
            optimizer.zero_grad()
            loss = F.binary_cross_entropy_with_logits(y_pred, y.unsqueeze(1))
            loss.backward()
            optimizer.step()
            sum_loss += loss.item()*y.shape[0]
            total += y.shape[0]
            idx += 1
        val_loss, val_acc = val_metrics(model, val_dl)
        if i % 5 == 1:
            print("train loss %.3f val loss %.3f and val accuracy %.3f" % (sum_loss/total, val_loss, val_acc))

        if best_val_loss > val_loss:
            best_val_loss = val_loss
            path = "model_{0}_loss_{1:.3f}.pth".format('quora', val_loss) 
            save_model(model, path)
            print(path)
    print("Best valid loss: {:.3f}".format(best_val_loss))


In [0]:
def val_metrics(model, valid_dl):
    model.eval()
    correct = 0
    total = 0
    sum_loss = 0.0
    for x1, x2, y in valid_dl:
        x1 = x1.long().cuda()
        x2 = x2.long().cuda()
        y = y.float().unsqueeze(1).cuda()
        y_hat = model(x1, x2)
        loss = F.binary_cross_entropy_with_logits(y_hat, y)
        y_pred = y_hat > 0
        correct += (y_pred.float() == y).float().sum()
        total += y.shape[0]
        sum_loss += loss.item()*y.shape[0]
    return sum_loss/total, correct/total

In [0]:
batch_size = 3000
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size=batch_size)
test_dl = DataLoader(test_ds, batch_size=batch_size)


In [0]:
vocab_size = len(pretrained_weight)
model = GRUModel(vocab_size, 50, 50,glove_weights=pretrained_weight).cuda()

parameters = filter(lambda p: p.requires_grad, model.parameters())
optimizer = torch.optim.Adam(model.parameters(), lr=0.5)

In [115]:
train_epocs(model, optimizer, train_dl, valid_dl, epochs=5)

model_quora_loss_0.578.pth
train loss 0.572 val loss 0.597 and val accuracy 0.665
model_quora_loss_0.568.pth
model_quora_loss_0.559.pth
Best valid loss: 0.559


In [0]:
model.embeddings.weight.requires_grad = True

In [117]:
#update_optimizer(optimizer, lr=0.001)
train_epocs(model, optimizer, train_dl, valid_dl, max_lr = 0.01, epochs=100)

model_quora_loss_0.548.pth
train loss 0.540 val loss 0.540 and val accuracy 0.722
model_quora_loss_0.540.pth
model_quora_loss_0.539.pth
model_quora_loss_0.531.pth
model_quora_loss_0.526.pth
model_quora_loss_0.518.pth
train loss 0.501 val loss 0.518 and val accuracy 0.737
model_quora_loss_0.518.pth
model_quora_loss_0.508.pth
model_quora_loss_0.504.pth
model_quora_loss_0.500.pth
train loss 0.462 val loss 0.496 and val accuracy 0.752
model_quora_loss_0.496.pth
model_quora_loss_0.484.pth
model_quora_loss_0.482.pth
model_quora_loss_0.481.pth
train loss 0.423 val loss 0.493 and val accuracy 0.754
model_quora_loss_0.464.pth
train loss 0.389 val loss 0.485 and val accuracy 0.767
model_quora_loss_0.460.pth
train loss 0.361 val loss 0.467 and val accuracy 0.790
train loss 0.339 val loss 0.481 and val accuracy 0.789
train loss 0.319 val loss 0.488 and val accuracy 0.794
train loss 0.304 val loss 0.491 and val accuracy 0.800
train loss 0.292 val loss 0.508 and val accuracy 0.800
train loss 0.281 v

In [0]:
load_model(model, 'model_quora_loss_0.460.pth')

In [0]:
def test_score(model, test_dl):
    model.eval()
    m = nn.Sigmoid()
    y_hat = []
    for x1, x2 in test_dl:
        x1 = x1.long().cuda()
        x2 = x2.long().cuda()
        out = m(model(x1, x2)).cuda()
        y_hat.append(out.detach().cpu().numpy())
    
    y_hat = np.vstack(y_hat)
    return y_hat

In [0]:
test_y = test_score(model, test_dl)

In [0]:
test_df['is_duplicate'] = test_y

In [0]:
submission = test_df[['test_id','is_duplicate']]
#submission.set_index()

In [0]:
submission.to_csv('submission.csv', index=False)

In [111]:
submission

,test_id,is_duplicate
0,0,0.588951
1,1,0.005010
2,2,0.396398
3,3,0.036706
4,4,0.248485
...,...,...
3563470,2345791,0.005506
3563471,2345792,0.022244
3563472,2345793,0.029853
3563473,2345794,0.178204


In [119]:
import csv
submission.drop(submission.tail(1217679).index,inplace=True) # drop last n rows
submission.to_csv('submmision_fixed.csv', index=False, quoting=csv.QUOTE_ALL)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
